### Business problem:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.
We will be provided a company name and their primary website.

### Business applications:

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more.

***Model used:-*** gpt-4o-mini

This is perhaps the example of Agentic AI design patterns, as we combined multiple calls to LLMs.

In [29]:
#Imports

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [3]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [9]:
# A class to represent a Webpage

headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

## First step: Have GPT-4o-mini figure out which links are relevant (First LLM call)

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

In [6]:
# System prompt
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [5]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
# User prompt
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [11]:
#ws = Website("https://huggingface.co")
#print(get_links_user_prompt(ws))

In [13]:
# Get only the relevant links from GPT
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [14]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}]}

## Second step: make the brochure! (Second LLM call)

Assemble all the details into another prompt to GPT4-o

In [16]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [26]:
#print(get_all_details("https://huggingface.co"))

In [18]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."


In [19]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [28]:
#get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

In [24]:
def create_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [25]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'discussion page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Company Brochure

**Welcome to Hugging Face**  
_Hugging Face is the AI community building the future._ Our platform provides an unparalleled collaborative environment for machine learning enthusiasts to create, discover, and share various models, datasets, and applications.

## Company Overview
Hugging Face serves as the leading hub for machine learning innovations, uniting over 50,000 organizations and thousands of contributors around a common goal: to democratize AI development. Our commitment to open-source principles ensures that everyone can access advanced machine learning tools and features, enabling a vibrant and growing community. 

---

## What We Offer
### Models & Datasets
- Explore **1M+ models** and **250k+ datasets** covering a variety of modalities including text, images, audio, and even 3D. 
- Collaborate on public ML models and datasets for enhanced productivity and innovation.

### Spaces
- Create and host applications that bring your ML ideas to life with our easy-to-use infrastructure.

### Enterprise Solutions
- We cater to organizations of all sizes with enterprise-grade features, including dedicated support, security protocols, and access controls. 
  - Pricing starts at **$20/user/month** for corporate teams.

### Open Source Commitment
- With our community-driven open-source initiatives, Hugging Face continues to push the boundaries of machine learning. 
- Our repositories include:
  - **Transformers** for PyTorch, TensorFlow, and JAX
  - **Diffusers** for modern image and audio generation
  - **Tokenizers** for fast text processing

---

## Company Culture
At Hugging Face, collaboration and innovation define our culture. We believe in building an inclusive and diverse workplace where everyone's contributions are valued. Our community-centric approach encourages open discussions and knowledge sharing, creating a supportive environment for personal and professional growth.

---

## Join Our Community
### Careers at Hugging Face
Become a part of our mission to democratize AI! We are looking for passionate and innovative individuals to join our team. Our openings range from engineering and research positions to roles in community management and marketing. Embrace a career where you can contribute to cutting-edge AI technologies and collaborate with like-minded professionals.

Explore available job opportunities on our [Careers Page](#).

---

## Our Customers
More than 50,000 organizations leverage Hugging Face products, including tech giants like:
- Amazon Web Services
- Google
- Microsoft
- Intel

Join a thriving ecosystem that includes both enterprises and passionate individuals who are shaping the future of AI.

---

## Connect with Us
Stay updated with our latest innovations and community activities!  
Follow us on:
- [Twitter](#)
- [LinkedIn](#)
- [GitHub](#)
- [Discord](#)

**Hugging Face** is more than just a company; we are a vibrant community dedicated to sharing knowledge, exploring new technologies, and building the future of machine learning together. Join us on this exciting journey!

--- 

For more information, visit our website at [Hugging Face](https://huggingface.co).